In [341]:
import psycopg2
import pandas as pd
import re
import os
from dotenv import load_dotenv

In [342]:
load_dotenv('.env')
HOST_NAME = os.environ.get('HOST_NAME')
DB_NAME = os.environ.get('DB_NAME')
USER_NAME = os.environ.get('USER_NAME')
PASSWORD = os.environ.get('PASSWORD')

In [343]:
conn = psycopg2.connect(host=HOST_NAME, user=USER_NAME, password=PASSWORD, database=DB_NAME)

In [344]:
users = pd.read_sql('select * from users',conn)
trades = pd.read_sql('select * from trades',conn)
conn.close()

In [346]:
users.dtypes

login_hash      object
server_hash     object
country_hash    object
currency        object
enable           int64
dtype: object

In [349]:
#object to string
users[['login_hash','server_hash','country_hash','currency']] = users[['login_hash','server_hash','country_hash','currency']].astype(str)

In [347]:
trades.dtypes

login_hash              object
ticket_hash             object
server_hash             object
symbol                  object
digits                   int64
cmd                      int64
volume                   int64
open_time       datetime64[ns]
open_price             float64
close_time      datetime64[ns]
contractsize           float64
dtype: object

In [350]:
#object to stirng
trades[['login_hash','ticket_hash','server_hash','symbol']] = trades[['login_hash','ticket_hash','server_hash','symbol']].astype(str)

In [351]:
users.head(5)

,login_hash,server_hash,country_hash,currency,enable
0,03A15F382AF3D505592CA69EFD4DC431,76791A893BDB9A68E8EE44898D581A60,AE54A5C026F31ADA088992587D92CB3A,USD,1
1,F8ADF5FF9840712CE2AF7FA9EA2A3FA7,76791A893BDB9A68E8EE44898D581A60,AE54A5C026F31ADA088992587D92CB3A,USD,1
2,B78CD788AB20A5D61CB13BD5389CABB3,76791A893BDB9A68E8EE44898D581A60,AE54A5C026F31ADA088992587D92CB3A,USD,1
3,DEFB729116227EF8582BFEC65F998489,76791A893BDB9A68E8EE44898D581A60,103357E3E40A9C0E4E9D36110F7BBC7A,USD,1
4,9EAD9A73AA6C93823C56BB8C2E798B45,76791A893BDB9A68E8EE44898D581A60,AE54A5C026F31ADA088992587D92CB3A,USD,1


In [352]:
trades.head(5)

,login_hash,ticket_hash,server_hash,symbol,digits,cmd,volume,open_time,open_price,close_time,contractsize
0,0DF500A57244E5B1670277767BEBB554,B321DC57F3B410507B986DD8962F8B1C,2F1CA40D76D7E6165DBA3924CE0D3B5C,EURUSD,5,1,5,2020-08-19 12:19:11,1.19459,2020-08-19 12:48:09,100000.0
1,6D66E192834605522190E51BF226FF5B,F68DA930EE1129D94EDFA02EBAF9E7B4,2F1CA40D76D7E6165DBA3924CE0D3B5C,XAUUSD,2,1,1,2020-08-28 03:09:02,1925.48000,2020-08-28 06:03:45,100.0
2,04852EC1A7D9FB8A291A24DDCE8C62B9,E351B480079ADEC7F6E6BE3ACEEA6EDF,2F1CA40D76D7E6165DBA3924CE0D3B5C,EURGBP,5,0,10,2020-08-25 13:47:34,0.90157,2020-09-08 12:20:43,100000.0
3,E62BE617269467931F2239C2771605C2,363723A73A926A7434E815F2F1D5ED61,2F1CA40D76D7E6165DBA3924CE0D3B5C,XAUUSD,2,1,10,2020-08-28 02:46:57,1926.68000,2020-08-28 02:51:13,100.0
4,54D2B4C5ECAE5FBC316524329846F81E,9CD4110328DA4CD71FFE0BE486D51094,2F1CA40D76D7E6165DBA3924CE0D3B5C,XAUUSD,2,1,85,2020-08-11 09:27:28,1999.12000,2020-08-11 09:27:54,100.0


### Please look for unexpected strings, unexpected numerical values, and unexpected dates. 

conclusions:

1. users table has duplications

2. 36 trades has open_time no smaller than close_time

3. 7 trades have no contractsize value

4. 11411 open_price have inconsistent digits

5. found some unexpected character in symbol e.g "USD,CHF"

6. found 1 trade with volume = 0

#### 1. check duplicated user 

In [353]:
users.columns

Index(['login_hash', 'server_hash', 'country_hash', 'currency', 'enable'], dtype='object')

In [354]:
users.duplicated().any()

True

In [355]:
#drop duplicates
users.drop_duplicates(inplace=True)

#### 2. check if open_time is smaller than close_time

In [356]:
# check open_time >= close_time
time_inconsistent_trades = trades[trades['open_time'] >= trades['close_time']]
print(f'found {len(time_inconsistent_trades)} trades with open_time larger than or equal to close_time')
time_inconsistent_trades.head(2)

found 36 trades with open_time larger than or equal to close_time


,login_hash,ticket_hash,server_hash,symbol,digits,cmd,volume,open_time,open_price,close_time,contractsize
839,08ECA1BC6F7779D852729F929080E6D8,6332934079F7D7361957A64F03A0C8DF,3D1F7E00251C43107EF39F55300781DB,EURUSD,5,1,3,2020-08-07 16:24:08,1.17796,2020-08-07 16:24:08,100000.0
3117,D1AE1E57F33114C8F162F65C0ADB38E6,68ED9A4538ACEA08AE3E14F297F3FE8A,2F1CA40D76D7E6165DBA3924CE0D3B5C,XAUUSD,2,0,10,2020-08-12 04:51:49,1896.18000,2020-08-12 04:51:49,100.0


#### 3. Column Null Check

In [357]:
#null check
trades.isnull().any(axis=0)

login_hash      False
ticket_hash     False
server_hash     False
symbol          False
digits          False
cmd             False
volume          False
open_time       False
open_price      False
close_time      False
contractsize     True
dtype: bool

In [358]:
non_contractsize_trades = trades[trades['contractsize'].isnull()]
print(f'found {len(non_contractsize_trades)} trades without contractsize value')
non_contractsize_trades.head(2)

found 7 trades without contractsize value


,login_hash,ticket_hash,server_hash,symbol,digits,cmd,volume,open_time,open_price,close_time,contractsize
869,2BCA9F6675B4F907E51DE5570E443D2B,B883EA07D74C0EB619A7ECC92F59312C,2F1CA40D76D7E6165DBA3924CE0D3B5C,COFFEE,2,1,100,2020-08-19 15:46:03,121.37,2020-08-19 16:26:32,NaN
6752,32DE9EECD10691C33D28F716CC255364,7EB45147326758A1128032D6D67E2235,3D1F7E00251C43107EF39F55300781DB,COFFEE,2,0,100,2020-08-11 16:33:22,112.89,2020-08-12 16:51:29,NaN


#### 4. check if "digit" field align with read digits of open_price

In [382]:
def significant_digits(x:str)->int:
    if '.' in x:
        digits = x.split('.')[1]
        num_digits = len(digits.rstrip('0'))
        return num_digits
    return 0
trades['num_digits'] = trades['open_price'].apply(lambda x:significant_digits(str(x)))
digits_inconsistent_trade = trades[trades['num_digits']!=trades['digits']]
print(f'found {len(digits_inconsistent_trade)} trades with inconsistent open_price digits')
digits_inconsistent_trade.head(2)

found 11411 trades with inconsistent open_price digits


,login_hash,ticket_hash,server_hash,symbol,digits,cmd,volume,open_time,open_price,close_time,contractsize,num_digits
9,275B715D599E43F0D6333F79AE4F117D,39B33A606DDA5BD4C2F78A102AA29E5F,2F1CA40D76D7E6165DBA3924CE0D3B5C,XAUUSD,2,0,12,2020-08-26 14:55:28,1916.9000,2020-08-26 15:15:32,100.0,1
25,18D4C2E739573770F9DF198F0E51C1B9,90E74D3BFAEB3FA020A45C5B68FCADBE,3D1F7E00251C43107EF39F55300781DB,AUDUSD,5,1,1,2020-08-24 17:00:17,0.7175,2020-08-24 17:56:28,100000.0,4


#### 5. check unexpected character in "symbol"

In [383]:
pattern = r'^[A-Z0-9]+$' #symbol should only contains uppper letters and numbers
invalid_symbol_trades = trades[~trades['symbol'].apply(lambda x: bool(re.match(pattern, x)))]
print(f'found {len(invalid_symbol_trades)} trades with invalid symbols')
invalid_symbol_trades.head(2)

found 2 trades with invalid symbols


,login_hash,ticket_hash,server_hash,symbol,digits,cmd,volume,open_time,open_price,close_time,contractsize,num_digits
33786,5D526301F0CCAD3D917C7EB483496B44,0813940283EF6306901B07A06C62AA21,2F1CA40D76D7E6165DBA3924CE0D3B5C,"USD,CHF",5,1,7,2020-08-03 17:42:00,0.92382,2020-08-03 17:58:19,100000.0,5
96877,D0C0CB6E5327AA037390AB5B4CED57AF,E726FCFB16769C2712FEF8CA61F1998D,2F1CA40D76D7E6165DBA3924CE0D3B5C,"USD,CHF",5,1,7,2020-08-03 17:42:00,0.92382,2020-08-03 17:58:19,100000.0,5


#### 6. check if volume is less than or equal to zero

In [362]:
zero_volume_trades = trades[trades['volume']==0]
print(f'found {len(zero_volume_trades)} trades with volume equal to 0')
zero_volume_trades.head(2)

found 1 trades with volume equal to 0


,login_hash,ticket_hash,server_hash,symbol,digits,cmd,volume,open_time,open_price,close_time,contractsize,num_digits
8349,9D6DFBCA3FD40AFF3097964853627A1E,BEA18D06F216DC9352B32B6CEDE0C373,54203B42716FE7C40138AE6C4913EBBC,EURUSD,5,0,0,2020-08-07 01:05:46,1.18804,2020-08-07 23:27:37,100000.0,5


## Please check that any joins you need to make to ensure data integrity.

conclusions:

1. 5026 users in trades table don't have record in users table

2. 570 trades are made by users whose accounts are not enabled

In [386]:
users.columns, trades.columns

(Index(['login_hash', 'server_hash', 'country_hash', 'currency', 'enable'], dtype='object'),
 Index(['login_hash', 'ticket_hash', 'server_hash', 'symbol', 'digits', 'cmd',
        'volume', 'open_time', 'open_price', 'close_time', 'contractsize',
        'num_digits'],
       dtype='object'))

#### 1. users and trades have common key login_hash,server_hash, check if they are same pairs in "users" and "trades"

In [387]:
merged = pd.merge(users,trades,on = ['login_hash'],how ='inner')
unpaired_df = merged[merged['server_hash_x'] != merged['server_hash_y']]
print(f'found {len(unpaired_df)} unpaired login_hash and server_hash')

found 0 unpaired login_hash and server_hash


#### 2. check login_hash in trade are all in users table

In [388]:
merged = pd.merge(users,trades,on = ['login_hash','server_hash'],how ='outer',indicator=True)
merged.groupby('_merge')['login_hash'].nunique()

_merge
left_only      550
right_only    5026
both           116
Name: login_hash, dtype: int64

In [389]:
# 550 users don't have any trading record
# 5026 users have at least one trade record but cannot be found in "users" table 
# only 116 users can be found in both "users" table and "trades" table

#### 3. Check if trades table contains users whose account is not enabled

In [390]:
unauthorized_trades = merged[(merged['_merge']=='both')&(merged['enable']==0)]   #euivalent to inner join on login_hash and server_hash and enable = 0
print('found',len(unauthorized_trades), 'trades where the users account is not enabled')
unauthorized_trades.head(2)

found 570 trades where the users account is not enabled


,login_hash,server_hash,country_hash,currency,enable,ticket_hash,symbol,digits,cmd,volume,open_time,open_price,close_time,contractsize,num_digits,_merge
1243,7B91CFD48D9B0A61C9720E35EA7317AB,3D1F7E00251C43107EF39F55300781DB,551FE18EF47D4E6E9D943B9A68ADA21D,USD,0.0,0A0DD0241FC1B28277E0FFD759E4E92B,XAUUSD,2.0,0.0,1.0,2020-08-05 11:54:13,2035.02,2020-08-05 12:45:41,100.0,2.0,both
1244,7B91CFD48D9B0A61C9720E35EA7317AB,3D1F7E00251C43107EF39F55300781DB,551FE18EF47D4E6E9D943B9A68ADA21D,USD,0.0,9AEA94E206E317BE5D247C17C8A30031,XAUUSD,2.0,0.0,1.0,2020-08-12 16:37:57,1937.58,2020-08-12 16:43:48,100.0,2.0,both


### Please test any edge cases you think should be investigated to produce the highest quality possible. 

#### 1. check if cmd value has only 1 and 0 

In [391]:
trades[~trades['cmd'].isin([0,1])]

,login_hash,ticket_hash,server_hash,symbol,digits,cmd,volume,open_time,open_price,close_time,contractsize,num_digits


#### 2. check if open_price is larger than zero

In [392]:
trades[trades['open_price'] < 0]

,login_hash,ticket_hash,server_hash,symbol,digits,cmd,volume,open_time,open_price,close_time,contractsize,num_digits


#### 3. check if currency is valid

In [393]:
currmap = {'USD', 'AUD', 'EUR', 'NZD'} # for example
users[~users['currency'].isin(currmap)]

,login_hash,server_hash,country_hash,currency,enable
